In [0]:
from pyspark.sql.types import *
from datetime import datetime

Curr_Year = datetime.now().year

schema = StructType([
        StructField('CAI_MARKET_FDM_SK', StringType(), False),
        StructField('DT_INVOICE', TimestampType(), False),
        StructField('DT_BORD_REF', TimestampType(), False),
        StructField('COMPANY_CODE', StringType(), False),
        StructField('COUNTRY_CODE', StringType(), False),
        StructField('CUSTOMER_CODE', StringType(), False),
        StructField('CAI_CODE', StringType(), False),
        StructField('PRODUCT_LINE_CODE', StringType(), False),
        StructField('QUANTITY', DoubleType(), False),
        StructField('CA_BF_EUROS', DoubleType(), False),
        StructField('CA_BF_CUR', DoubleType(), False),
        StructField('INVOICE_CURRENCY', StringType(), False),
        StructField('MT_HT_EUROS', DoubleType(), False),
        StructField('MT_HT_CUR', DoubleType(), False),
        StructField('BRAND_CODE', StringType(), False),
        StructField('MARKET_CODE', StringType(), False),
        StructField('DT_UPDATE', TimestampType(), False),
        StructField('FILE_AUDIT_ID', StringType(), False),
        StructField('FILE_ID', StringType(), False),
        StructField('DT_YEARMONTH', StringType(), False),
        StructField('TONNAGE', DoubleType(), False),
        StructField('MFS_IDR', StringType(), False),
        StructField('ORIGINAL_COMPANY_C', StringType(), False),
        StructField('OFF_RCHP_C', StringType(), False),
        StructField('SOURCE_C', StringType(), False),
        StructField('CUST_FDM_SK', StringType(), False),
        StructField('DT_BORDEREAU', TimestampType(), False)
        ])

path = ["/mnt/adls/30_datamart_D2/KBI/Daily_Sales/FULL/FDM_DAILY_SALES_WW_F-"+str(Curr_Year)+"*.csv", 
        "/mnt/adls/30_datamart_D2/KBI/Daily_Sales/FULL/FDM_DAILY_SALES_WW_F-"+str(Curr_Year-1)+"*.csv",
        "/mnt/adls/30_datamart_D2/KBI/Daily_Sales/FULL/FDM_DAILY_SALES_WW_F-"+str(Curr_Year-2)+"*.csv",
        "/mnt/adls/30_datamart_D2/KBI/Daily_Sales/FULL/FDM_DAILY_SALES_WW_F-"+str(Curr_Year-3)+"*.csv",
        "/mnt/adls/30_datamart_D2/KBI/Daily_Sales/FULL/FDM_DAILY_SALES_WW_F-"+str(Curr_Year-4)+"*.csv"]

Daily_Sales = (spark.read
               .option('header', 'true')
               .option('sep', ";")
               .schema(schema)
               .csv(path))

Daily_Sales.write.format("delta").mode("overwrite").partitionBy("DT_BORD_REF").save("/mnt/kbi/KBI_NG/PROD/tmp/Daily_Sales")
display(spark.sql("DROP TABLE  IF EXISTS Daily_Sales"))
display(spark.sql("CREATE TABLE Daily_Sales USING DELTA LOCATION '/mnt/kbi/KBI_NG/PROD/tmp/Daily_Sales'"))   
display(spark.sql("OPTIMIZE Daily_Sales ZORDER BY (CUSTOMER_CODE)"))

In [0]:
import pyspark.sql.functions as F

path_Ref_Geo = '/mnt/adls/20_datastore/CT2/Geography/FDM_COUNTRY_CUR_REG_CMN_DIM.csv'

Ref_Geo = (spark.read.csv(path_Ref_Geo, sep=";", header=True)
          .select(F.col("COUNTRY_MICHELIN_CODE").alias("COUNTRY_CODE"), F.col("COUNTRY_CODE").alias("COUNTRY_ALPHA")))

Ref_Geo.write.format("delta").mode("overwrite").partitionBy("COUNTRY_CODE").save("/mnt/kbi/KBI_NG/PROD/tmp/Ref_Geo")

display(spark.sql("DROP TABLE  IF EXISTS Ref_Geo"))
display(spark.sql("CREATE TABLE Ref_Geo USING DELTA LOCATION '/mnt/kbi/KBI_NG/PROD/tmp/Ref_Geo'"))   
display(spark.sql("OPTIMIZE Ref_Geo ZORDER BY (COUNTRY_CODE)"))